# Explore extracting melody
Since Fri. Nov. 23rd, 2021


`music21` has better support on extracting melody, given low-level music note support,
and it seems to work better with Music XML files.



## Setup



In [1]:
from icecream import ic

from util import *
from util import MidoUtil as Mu, PrettyMidiUtil as Pmu
from melody_extractor import MelodyExtractor


songs = eg_songs(fmt='MXL')
ic(songs)



ic| eg_songs(fmt='MXL'): ['/Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MXL-eg/Joe '
                          'Hisaishi - Merry Go Round of Life (bitmidi).mxl']


['/Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MXL-eg/Joe Hisaishi - Merry Go Round of Life (bitmidi).mxl']

## Get the bar with maximum average pitch


### Merry Go Round of Life


In [2]:
fnm = eg_songs(k='Merry Go Round', fmt='MXL')
ic(fnm)




ic| fnm: ('/Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MXL-eg/Joe '
          'Hisaishi - Merry Go Round of Life (bitmidi).mxl')


'/Users/stefanh/Documents/UMich/Research/Music with NLP/datasets/MXL-eg/Joe Hisaishi - Merry Go Round of Life (bitmidi).mxl'